# Estimation of Transformation between sets of coordinates

## Input

- two datasets with tables of coordinates (in physical units) to match

## Output

- two JSON files describing a **global transformation** from moving coordinates to target coordinates and **local transformations** for each image in the moving dataset

In [2]:
from pathlib import Path
import json

import pandas as pd
import numpy as np
from skimage.transform import SimilarityTransform, EuclideanTransform
from skimage.measure import ransac

from calmutils.descriptors import descriptor_local_qr, match_descriptors_kd
from utils.transform_helpers import affine_transform_nd

## Parameters

In [7]:
# base paths of target and moving dataset
base_path_target = "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/"
base_path_moving = "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/RNAFISH/"

# paths of coordinate tables relative to base paths
# NOTE: can be paths to a single .csv file, but can also include wildcard (*) to combine multiple files
coordinates_path_target = 'detections_beads/merge_global_coords.csv'
coordinates_path_moving = 'detections_beads/merge_global_coords.csv'

# subdirectory of moving dataset to save transform parameters to
save_subdir = 'alignment_parameters'

# name of world coordinates and 
coordinate_column_names = ['z_global_um', 'y_global_um', 'x_global_um']
image_file_column_name = 'img'

# in order of complexity:
# euclidean: move & rotate, similarity: + scale, affine: + shear
transform_type = 'similarity'

# matching & RANSAC parameters
# NOTE: for global, we do more rounds and use more lenient threshold
descriptor_match_ratio = 2.0
min_samples_ransac = 4
residual_thresh_global = 10.0
residual_thresh_local = 4.0
ransac_rounds_global = 100_000
ransac_rounds_local = 20_000

# when doing local alignment, radius around center of moving image to consider (in um)
match_radius = 25.0

n_neighbors_descriptor = 4
# descriptor redundancy
# NOTE: will quickly become much slower when this value is increased
descriptor_redundancy = 0

## 1) Global Alignment

First, we will load all fiducial coordinates for the moving and target dataset and try to find a geometric transform between them.

**Make sure this works** (i.e. you have a reasonable amount of matches and inliers), as this is required for a local alignment (step 2) 

In [4]:
# load tables
df_target = pd.concat([pd.read_csv(f) for f in Path(base_path_target).glob(coordinates_path_target)])
df_moving = pd.concat([pd.read_csv(f) for f in Path(base_path_moving).glob(coordinates_path_moving)])

# get coordinates from tables
coords_target = df_target[coordinate_column_names].values
coords_moving = df_moving[coordinate_column_names].values

# generate descriptors
desc_target, idxs_target = descriptor_local_qr(coords_target, redundancy=descriptor_redundancy, n_neighbors=n_neighbors_descriptor, progress_bar=True)
desc_moving, idxs_moving = descriptor_local_qr(coords_moving, redundancy=descriptor_redundancy, n_neighbors=n_neighbors_descriptor, progress_bar=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1534/1534 [00:00<00:00, 6298.19it/s]


### Estimate Transform

In [5]:
# match descriptors and select matching coords
matches_kd = match_descriptors_kd(desc_target, desc_moving, max_ratio=1/descriptor_match_ratio, cross_check=True)
coords_target_match = coords_target[idxs_target[matches_kd.T[0]]]
coords_moving_match = coords_moving[idxs_moving[matches_kd.T[1]]]

# select transform based on name from parameters
transform_type_class = {'euclidean': EuclideanTransform, 'similarity': SimilarityTransform, 'affine': affine_transform_nd(3)}[transform_type]

# estimate transform with RANSAC
transform_global, inliers_global = ransac((coords_moving_match, coords_target_match),
                            transform_type_class, min_samples_ransac, residual_threshold=residual_thresh_global, max_trials=ransac_rounds_global,
                            stop_probability=1)

# get some metrics and print
num_matches_global = len(coords_moving_match)
num_inliers_global = inliers_global.sum()
mean_error_global = np.linalg.norm(coords_target_match[inliers_global] - transform_global(coords_moving_match[inliers_global]), axis=1).mean()

print(f'{num_inliers_global} inliers of {num_matches_global} matches ({100 * num_inliers_global / num_matches_global :.2f} %).')
print(f'mean error of matched points: {mean_error_global :.2f} µm.')

48 inliers of 60 matches (80.00 %).
mean error of matched points: 0.60 µm.


### Save global results

In [6]:
global_results = {
    'transformations': [
        {
            'center_coords': list(map(float, coords_moving.mean(axis=0))),
            'num_matches': num_matches_global,
            'num_inliers': int(num_inliers_global),
            'mean_error': float(mean_error_global),
            'transform_type': transform_type,
            'parameters': list(map(float, transform_global.params.flat))
        }
    ]
}

if not (Path(base_path_moving) / save_subdir).exists():
    (Path(base_path_moving) / save_subdir).mkdir()
with open(Path(base_path_moving) / save_subdir / 'alignment_parameters_global.json', 'w') as fd:
    json.dump(global_results, fd, indent=1)